# 2.- Partición Externa de Datos Externa (Hold-Out)
Se va a comenzar realizando la partición de datos externa para separar el conjunto de datos total en los subconjuntos de entrenamiento (train), y prueba (test).
Dado que estamos ante un problema de series temporales, se requiere mantener la coherencia temporal por lo que:

* No se aplicará shuffle a los datos.
* En este caso asegurar que datos de entrenamiento no se localizan entre los datos de test, no sólo se limita a las instancias individuales en sí, sino que dado que nuestro objetivo será predecir si en un cierto año y para un determinado país se está ante una posible situación de pre-crisis, también tenemos que asegurar que no existe información relativa a años posteriores en nuestro conjunto de test ni para el país bajo estudio en concreto ni tampoco para otros paises (*Hellwig, K.P. (2021). Predicting Fiscal Crises: A Machine Learning Approach*).
* Para cumplir lo anterior se particionará el dataset a partir de un año concreto manteniendo los valores más cercanos a una distribución de porcentajes igual al 80% para datos de entrenamiento y 20% para datos de prueba.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Cargamos nuestro dataset
data = pd.read_csv('../datasets/finales/data_final.csv', sep=';', na_values='', decimal=',')

Tendremos que tener en cuenta que las instancias para cada año no son iguales en el dataset final para mantener un porcentaje 80%-20% adecuado:

In [3]:
pct_train = 0.8

num_paises = len(np.unique(data['iso']))
print(f'Número Total de Países: {num_paises}')

num_anios = len(np.unique(data['year']))
print(f'Número Total de Años: {num_anios}')

instancias_test = data.shape[0] * (1 - pct_train)

anio_particion = int((data['year'].max() - (instancias_test/num_paises))) - 2
print(f'Año de Particionado: {anio_particion}')

train = data[data['year'] < anio_particion]
test = data[data['year'] >= anio_particion]

print(f'Muestras de Entrenamiento: {train.shape} -- {train.shape[0] * 100 / data.shape[0]:.2f}% del total')
print(f'Muestras de Test: {test.shape} -- {test.shape[0] * 100 / data.shape[0]:.2f}% del total')

etiquetas = ['No Crisis', 'Crisis']
_, ocurrencias_train = np.unique(train['crisisJST'], return_counts=True)
_, ocurrencias_test = np.unique(test['crisisJST'], return_counts=True)
print(f'Distribución de Muestras:\n\tEntrenamiento: {etiquetas} - {ocurrencias_train}\n\tTest: {etiquetas} - {ocurrencias_test}')

Número Total de Países: 18
Número Total de Años: 75
Año de Particionado: 2004
Muestras de Entrenamiento: (984, 14) -- 80.33% del total
Muestras de Test: (241, 14) -- 19.67% del total
Distribución de Muestras:
	Entrenamiento: ['No Crisis', 'Crisis'] - [960  24]
	Test: ['No Crisis', 'Crisis'] - [215  26]


In [4]:
# Guardo las particiones
import os
if not os.path.exists('../particiones'):
    os.mkdir('../particiones')

train.to_csv('../particiones/train.csv', sep=';', na_rep='', decimal=',', index=False)
test.to_csv('../particiones/test.csv', sep=';', na_rep='', decimal=',', index=False)